In [56]:
import json
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle
from util import compute_metrics, foresee
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


data = json.load(open('trainer_state2.json'))

In [37]:
mse = []
loss = []
mae = []
r2 = []
for i in range(len(data['log_history'])):
    mse.append(data['log_history'][i]['eval_mse'])
    loss.append(data['log_history'][i]['eval_loss'])
    mae.append(data['log_history'][i]['eval_mae'])
    r2.append(data['log_history'][i]['eval_r2'])

In [38]:
data['log_history'][0]

{'epoch': 1.0,
 'eval_loss': 0.5257409811019897,
 'eval_mae': 0.49442604184150696,
 'eval_mse': 0.5257410407066345,
 'eval_r2': 0.4223982891074357,
 'eval_rmse': 0.7250800132751465,
 'eval_runtime': 27.1235,
 'eval_samples_per_second': 15.78,
 'eval_steps_per_second': 0.811,
 'step': 27}

In [39]:
dataframe = pd.DataFrame(mse, columns=['MSE'])
dataframe['Loss'] = loss
dataframe['MAE'] = mae
dataframe['R2'] = r2

In [40]:
dataframe

,MSE,Loss,MAE,R2
0,0.525741,0.525741,0.494426,0.422398
1,0.549506,0.549506,0.491572,0.396289
2,0.489995,0.489995,0.468017,0.461671
3,0.462154,0.462154,0.454090,0.492258
4,0.466428,0.466428,0.456430,0.487562


In [41]:
px.line(dataframe)

In [75]:
model = AutoModelForSequenceClassification.from_pretrained('../models/trainer_auto_model')
tokenizer = AutoTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1')
scaler = pickle.load(open('../results/scalers/scaler_auto_model.pkl','rb'))

training_args = TrainingArguments(
    output_dir ='../results',
    num_train_epochs = 5,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 20,
    weight_decay = 0.01,
    learning_rate = 2e-5,
    logging_dir = '../logs',
    save_total_limit = 10,
    load_best_model_at_end = True,
    metric_for_best_model = 'rmse',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
)

# Call the Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics_for_regression,
)

loading configuration file ../models/trainer_auto_model\config.json
Model config BertConfig {
  "_name_or_path": "../models/trainer_auto_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "regression",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 60000
}

loading weights file ../models/trainer_auto_model\pytorch_model.bin
All model checkpoint weights were used when initiali

In [69]:
prediction = foresee(trainer, scaler, tokenizer, 'iphone 11 pro', (1, 90))

***** Running Prediction *****
  Num examples = 89
  Batch size = 8


In [86]:
px.scatter(prediction, trendline='lowess')